Credits for this code to:
- Pierre Megret https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook

The code is mainly based on his tutorial. <br>
The gender bias measurements are done as before. <br>
WEAT tests are performeds using wefe package.

# 0. Getting started

In [2]:
import re  # For preprocessing
from time import time  # To time our operations
from collections import defaultdict  # For word frequency
import multiprocessing

import numpy as np
import pandas as pd  # For data handling
import spacy  # For preprocessing
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import wefe
from wefe.query import Query
from wefe.utils import load_test_model
from wefe.metrics import WEAT
from wefe.word_embedding_model import WordEmbeddingModel

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [3]:
df = pd.read_csv('../data/alicewu_posts/gendered_posts.csv')
print(df.shape)
df.head()

(444810, 16)


,title_id,post_id,topic,raw_post,fem_all,male_all,training,ypred,female,fem_pronoun,male_pronoun,training_pronoun,ypred_pronoun,female_pronoun,time_latest,month_latest
0,1,4,request-a-thread-to-be-deleted-here,"The fires went out, and blank darkness fell. T...",0,7,0.0,0.001674,0,0,7,0.0,1.860989e-04,0.0,2 hours,0.0
1,1,5,request-a-thread-to-be-deleted-here,Mod here. The meaning of life is max V(A(t))=...,0,1,1.0,0.082022,0,0,1,1.0,8.067608e-03,0.0,2 hours,0.0
2,1,9,request-a-thread-to-be-deleted-here,The facts established in the recent paper by W...,0,1,1.0,0.032163,0,0,0,NaN,2.387353e-02,NaN,2 hours,0.0
3,1,11,request-a-thread-to-be-deleted-here,OBLANCHARD-assuming you are Olivier Blanchard ...,5,5,NaN,0.000792,0,0,0,NaN,2.526144e-02,NaN,2 hours,0.0
4,2,2,about-ejmr,Q: Why should I visit this site? A: There is a...,1,0,1.0,0.000008,1,0,0,NaN,2.388843e-12,NaN,6 days,0.0


In [11]:
# Prototyping on subset of the posts
subset = df.iloc[0:1000]
merged_text = subset['raw_post'].str.cat(sep=' ')

# 1. Preprocess the posts 

In [5]:
# Installing English language
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 12.8 MB 2.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/bryskulov/Desktop/QEM Barcelona/Thesis/thesis-gender-bias/env/bin/python3 -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
# Creating a spacy instance
nlp = spacy.load('en_core_web_sm')
nlp.max_length = 10000000

In [13]:
# Break the posts into seperate sentences

t = time()

doc = nlp(merged_text)
sents = [sent.text.strip() for sent in doc.sents]

print('Time to sentenize the posts: {} mins'.format(round((time() - t) / 60, 2)))

Time to sentenize the posts: 0.12 mins


In [9]:
# Preprocess all posts in the dataset

t = time()

lower_index = 0
sents_store = list()

for upper_index in range(0, 440000, 10000):
    print("Processing row #: ", upper_index)
    subset = df.iloc[lower_index:upper_index]
    merged_text = subset['raw_post'].str.cat(sep=' ')
    
    t = time()
    doc = nlp(merged_text)
    sents = [sent.text.strip() for sent in doc.sents]
    print('Time to sentenize the book: {} mins'.format(round((time() - t) / 60, 2)))
    
    sents_store.append(sents)
    lower_index = upper_index
    
print('Time to process the posts: {} mins'.format(round((time() - t) / 60, 2)))

In [ ]:
# Merge all the sentences
from functools import reduce
sents = reduce(lambda xs, ys: xs + ys, sents_store)

In [11]:
# Save the processed posts
import pickle
with open("forum_posts.pkl", "wb") as fp:   #Pickling
    pickle.dump(sents, fp)
    
# with open("test", "rb") as fp:   # Unpickling
#     b = pickle.load(fp)

In [138]:
with open("forum_posts.pkl", "rb") as fp:   # Unpickling
    sents = pickle.load(fp)

# 2. Clean the posts with Spacy 

In [14]:
#loading the english language small model of spacy
nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
stopwords = nlp.Defaults.stop_words

print(len(stopwords))

326


In [15]:
#Removing pronouns from spacy stopwords
nlp.Defaults.stop_words -= {'he', 'his', 'him', 'himself', 'male', 'man', 'men'}
nlp.Defaults.stop_words -= {'she', 'her', 'herself', 'female', 'woman', 'women'}

In [16]:
def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

In [17]:
# Removes non-alphabetic characters:
brief_cleaning = (re.sub("[^A-Za-z']+", ' ', str(sent)).lower() for sent in sents)

In [18]:
t = time()
txt = [cleaning(doc) for doc in nlp.pipe(brief_cleaning, batch_size=5000, n_process=-1)]

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

Time to clean up everything: 0.19 mins


In [19]:
# removing Nones and duplicates
txt = list(set(filter(None, txt)))

# 3. Identify bigrams

In [20]:
sent = [sent.split() for sent in txt]

In [21]:
phrases = Phrases(sent, min_count=30, progress_per=10000)

INFO - 19:24:36: collecting all words and their counts
INFO - 19:24:36: PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO - 19:24:36: collected 19257 token types (unigram + bigrams) from a corpus of 18233 words and 2297 sentences
INFO - 19:24:36: merged Phrases<19257 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 19:24:36: Phrases lifecycle event {'msg': 'built Phrases<19257 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.04s', 'datetime': '2022-06-21T19:24:36.781420', 'gensim': '4.2.0', 'python': '3.9.10 (v3.9.10:f2f3f53782, Jan 13 2022, 17:02:14) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [22]:
bigram = Phraser(phrases)
sentences = bigram[sent]

INFO - 19:24:38: exporting phrases from Phrases<19257 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000>
INFO - 19:24:38: FrozenPhrases lifecycle event {'msg': 'exported FrozenPhrases<0 phrases, min_count=30, threshold=10.0> from Phrases<19257 vocab, min_count=30, threshold=10.0, max_vocab_size=40000000> in 0.05s', 'datetime': '2022-06-21T19:24:38.409196', 'gensim': '4.2.0', 'python': '3.9.10 (v3.9.10:f2f3f53782, Jan 13 2022, 17:02:14) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [23]:
sentences[1]

['actually',
 'yeah',
 'weird',
 'thought',
 'relationship',
 'mean',
 'boring',
 'life']

## 3.2 Most frequent words

In [24]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

4763

In [25]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['his', 'like', 'guy', 'good', 'woman', 'think', 'paper', 'he', 'know', 's']

In [26]:
word_freq['she']

52

# 4. Train the Word2Vec model

In [27]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer

In [28]:
w2v_model = Word2Vec(min_count=10,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=25,
                     workers=cores-1)

INFO - 19:24:46: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=300, alpha=0.03>', 'datetime': '2022-06-21T19:24:46.512550', 'gensim': '4.2.0', 'python': '3.9.10 (v3.9.10:f2f3f53782, Jan 13 2022, 17:02:14) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'created'}


In [29]:
t = time()

w2v_model.build_vocab(sentences, progress_per=10000)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 19:24:47: collecting all words and their counts
INFO - 19:24:47: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 19:24:47: collected 4763 word types from a corpus of 18233 raw words and 2297 sentences
INFO - 19:24:47: Creating a fresh vocabulary
INFO - 19:24:47: Word2Vec lifecycle event {'msg': 'effective_min_count=10 retains 381 unique words (8.00% of original 4763, drops 4382)', 'datetime': '2022-06-21T19:24:47.347589', 'gensim': '4.2.0', 'python': '3.9.10 (v3.9.10:f2f3f53782, Jan 13 2022, 17:02:14) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare_vocab'}
INFO - 19:24:47: Word2Vec lifecycle event {'msg': 'effective_min_count=10 leaves 9084 word corpus (49.82% of original 18233, drops 9149)', 'datetime': '2022-06-21T19:24:47.348769', 'gensim': '4.2.0', 'python': '3.9.10 (v3.9.10:f2f3f53782, Jan 13 2022, 17:02:14) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'prepare

Time to build vocab: 0.0 mins


In [30]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

INFO - 19:24:47: Word2Vec lifecycle event {'msg': 'training model with 7 workers on 381 vocabulary and 300 features, using sg=0 hs=0 sample=6e-05 negative=25 window=2 shrink_windows=True', 'datetime': '2022-06-21T19:24:47.707155', 'gensim': '4.2.0', 'python': '3.9.10 (v3.9.10:f2f3f53782, Jan 13 2022, 17:02:14) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'train'}
INFO - 19:24:47: EPOCH 0: training on 18233 raw words (1432 effective words) took 0.0s, 44478 effective words/s
INFO - 19:24:47: EPOCH 1: training on 18233 raw words (1536 effective words) took 0.0s, 49444 effective words/s
INFO - 19:24:47: EPOCH 2: training on 18233 raw words (1481 effective words) took 0.0s, 49460 effective words/s
INFO - 19:24:47: EPOCH 3: training on 18233 raw words (1530 effective words) took 0.0s, 47011 effective words/s
INFO - 19:24:47: EPOCH 4: training on 18233 raw words (1477 effective words) took 0.0s, 50490 effective words/s
INFO - 19:24:47: EPOCH 5: traini

Time to train the model: 0.02 mins


In [31]:
w2v_model.init_sims(replace=True)

/var/folders/6m/7qj3x7j16pqbzfycpphbz9z40000gn/T/ipykernel_33101/514372312.py:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)
WARNING - 19:24:48: destructive init_sims(replace=True) deprecated & no longer required for space-efficiency


In [32]:
w2v_model.wv.most_similar(positive=["woman"])

[('find', 0.9997744560241699),
 ('play', 0.9997660517692566),
 ('paper', 0.9997659921646118),
 ('publish', 0.9997531771659851),
 ('job', 0.9997520446777344),
 ('wife', 0.9997513294219971),
 ('chinese', 0.999751091003418),
 ('html', 0.9997481107711792),
 ('r', 0.9997481107711792),
 ('bro', 0.9997469186782837)]

In [33]:
w2v_model.wv.most_similar(positive=["man"])

[('thread', 0.9997735619544983),
 ('html', 0.9997717142105103),
 ('her', 0.9997581243515015),
 ('show', 0.9997564554214478),
 ('tenure', 0.9997557401657104),
 ('male', 0.9997535943984985),
 ('professor', 0.9997529983520508),
 ('economic', 0.9997521638870239),
 ('level', 0.999751627445221),
 ('research', 0.9997478127479553)]

# 5. Measuring Gender Bias

## 5.1 Defining target and attribute words

In [34]:
female_pronouns = ['her', 'herself', 'she']
female_names = ['amy', 'athey', 'duflo', 'elizabeth', 'emily', 'hilary',
                'hillary', 'jane', 'jennifer', 'jessica', 'maria', 'mary',
                'nancy', 'reinhart', 'sarah', 'susan', 'yellen']
female_identity = ['female', 'females', 'ladies', 'lady', 'woman', 'women']
female_miscellaneous = ['bietches', 'bitch', 'bitches', 'broette', 'broettes', 'chick', 'chicks',
                'daughter', 'daughters', 'gf', 'girl', 'girlfriend', 'girlfriends',
                'girls', 'mom', 'moms', 'mother', 'mothers', 'sister', 'sisters', 'wife',
                'wives']

female_words = female_pronouns + female_names + female_identity + female_miscellaneous

In [35]:
male_pronouns = ['he', 'him', 'himself', 'his']
male_names = ['acemoglu', 'adam', 'akerlof', 'alan', 'albert', 'alexander', 'allen', 'andrew', 'angrist', 'arthur',
'autor', 'baker', 'barro', 'becker', 'ben', 'benjamin', 'bernanke', 'berry', 'blanchard', 'bob', 'borjas']
male_identity = ['male', 'males', 'man', 'men']
male_miscellaneous = ['bf', 'boy', 'boyfriend', 'boys', 'bro', 'bros', 'brother', 'brothers', 'dad', 'daddy',
'dude', 'dudes', 'father', 'fathers', 'gentleman', 'grandfather', 'guy', 'guys',
'husband', 'husbands', 'papa', 'sir', 'son', 'sons', 'uncle']

male_words = male_pronouns + male_names + male_identity + male_miscellaneous

In [36]:
male_names_full = ['acemoglu', 'adam', 'akerlof', 'alan', 'albert', 'alexander', 'allen', 'andrew', 'angrist', 'arthur',
'autor', 'baker', 'barro', 'becker', 'ben', 'benjamin', 'bernanke', 'berry', 'blanchard', 'bob', 'borjas',
'brian', 'campbell', 'carl', 'carlos', 'charlie', 'chetty', 'chris', 'christopher', 'cochrane', 'colin',
'cowen', 'daniel', 'daron', 'david', 'deaton', 'delong', 'duffie', 'edward', 'eric', 'eugene', 'fama',
'frank', 'frey', 'friedman', 'friedman', 'gary', 'george', 'gintis', 'glaeser', 'gordon', 'greene',
'greg', 'gregory', 'hansen', 'harry', 'hayashi', 'hayek', 'heckman', 'henderson', 'henry', 'imben',
'jack', 'james', 'jason', 'je', 'jerey', 'jeremy', 'jimmy', 'joe', 'john', 'jon', 'jonathan', 'jose',
'joseph', 'justin', 'kehoe', 'ken', 'kenneth', 'kevin', 'krueger', 'kruggles', 'krugman', 'krugmans',
'larry', 'lars', 'levine', 'levitt', 'lucas', 'mankiw', 'mark', 'martin', 'matt', 'matthew', 'michael',
'mike', 'miller', 'milton', 'murphy', 'myerson', 'neumann', 'nicholas', 'nick', 'noah', 'parag', 'pat',
'pathak', 'patrick', 'paul', 'perez', 'peter', 'phil', 'philip', 'phillips', 'pierre', 'piketty', 'pikettys',
'powell', 'prescott', 'rabin', 'raj', 'ravikumar', 'ricardo', 'richard', 'robert', 'roberts', 'robinson',
'roger', 'rogo', 'ron', 'roth', 'rubin', 'rubinstein', 'russ', 'rust', 'ryan', 'saez', 'sam', 'sargent', 'shapiro',
'shiller', 'shleifer', 'simon', 'sims', 'stephen', 'steve', 'steven', 'stiglitz', 'summers', 'terry', 'thaler',
'thomas', 'tim', 'tirole', 'tom', 'tony', 'victor', 'walker', 'wallace', 'walter', 'werning', 'williamson',
'wolfer', 'woodford', 'wooldridge']

In [37]:
# All Gender Classifiers
male_words = ['he', 'his', 'him', 'himself', 'male', 'man', 'men']
female_words = ['she', 'her', 'herself', 'female', 'woman', 'women']

In [38]:
# Academic/Professional
economics = ['economics', 'macro', 'empirical','qje', 'keynesian']
academic_general = ['research', 'papers','tenure', 'teaching', 'professor']
professional = ['career', 'interview', 'payrolls', 'placement', 'recruit']

In [39]:
# Personal/Physical
personal_information = ['family','married', 'kids', 'relationship','lifestyle']
physical_attributes = ['beautiful', 'handsome', 'attractive', 'body','fat']
gender_related = ['gender', 'feminine', 'masculine', 'sexist', 'sexual']

In [40]:
# Swear Words
swear = ['shit', 'wtf', 'asshole']

In [41]:
# Intellectual
intellectual_positive = ['intelligent', 'creative','competent']
intellectual_neutral = ['brain', 'iq', 'ability']
intellectual_negative = ['dumb','ignorant','incompetent']

In [42]:
# Miscellaneous
emotion_feelings = ['happy','depressing']
others = ['years', 'places', 'everything']

## 5.2 Defining functions for gender bias measurement

In [43]:
def average_attr_words(vectors, word_list1, word_list2):
    words_to_average1 = list()
    words_to_average2 = list()
    
    for word1 in word_list1:
        try:
            words_to_average1.append(vectors[word1])
        except:
            print("Word is not present: ", word1)
            
    for word2 in word_list2:
        try:
            words_to_average2.append(vectors[word2])
        except:
            print("Word is not present: ", word2)
            
    #print("The length of average word list 1: ", len(words_to_average1))
    #print("The length of average word list 2: ", len(words_to_average2))
    
    averaged_words1 = np.array(words_to_average1).mean(axis=0)
    averaged_words2 = np.array(words_to_average2).mean(axis=0)

    
    return averaged_words1, averaged_words2

In [44]:
def cossim(v1, v2, signed = True):
    c = np.dot(v1, v2)/np.linalg.norm(v1)/np.linalg.norm(v2)
    if not signed:
        return abs(c)
    return c

In [46]:
def calc_distance_between_vectors(vec1, vec2, distype = 'norm'):
    if distype == 'norm':
        return np.linalg.norm(np.subtract(vec1, vec2))
    else:
        return cossim(vec1, vec2)

In [47]:
def calc_relative_norm_distance(vectors, male_word_list, female_word_list, neutral_words):
    
    male_avg_vec, female_avg_vec = average_attr_words(vectors, male_word_list, female_word_list)
    
    list_rel_norm_dist = []
    for word in neutral_words:
        try:
            rel_norm_dist = calc_distance_between_vectors(vectors[word], male_avg_vec) - \
                            calc_distance_between_vectors(vectors[word], female_avg_vec)
            list_rel_norm_dist.append(rel_norm_dist)
        except:
            print("Word is not present: ", word)
    return np.array(list_rel_norm_dist).mean()

# 6. Gender Bias measurement following Garg et al. 

In [48]:
group_of_words = [economics, academic_general, professional, personal_information, physical_attributes, 
                  gender_related, intellectual_positive, intellectual_neutral, intellectual_negative]

## On gender pronouns

In [49]:
calc_relative_norm_distance(w2v_model.wv, male_pronouns, female_pronouns, academic_general + professional)

Word is not present:  him
Word is not present:  herself
Word is not present:  papers
Word is not present:  teaching
Word is not present:  interview
Word is not present:  payrolls
Word is not present:  placement
Word is not present:  recruit


-0.0006682966

In [50]:
calc_relative_norm_distance(w2v_model.wv, male_pronouns, female_pronouns, personal_information + physical_attributes)

Word is not present:  him
Word is not present:  herself
Word is not present:  married
Word is not present:  kids
Word is not present:  relationship
Word is not present:  lifestyle
Word is not present:  beautiful
Word is not present:  handsome
Word is not present:  attractive
Word is not present:  fat


-0.00025895052

## On gender names

In [51]:
calc_relative_norm_distance(w2v_model.wv, male_names, female_names, economics)

Word is not present:  acemoglu
Word is not present:  adam
Word is not present:  akerlof
Word is not present:  alan
Word is not present:  albert
Word is not present:  alexander
Word is not present:  allen
Word is not present:  andrew
Word is not present:  angrist
Word is not present:  arthur
Word is not present:  autor
Word is not present:  baker
Word is not present:  barro
Word is not present:  becker
Word is not present:  ben
Word is not present:  benjamin
Word is not present:  bernanke
Word is not present:  berry
Word is not present:  blanchard
Word is not present:  bob
Word is not present:  borjas
Word is not present:  amy
Word is not present:  athey
Word is not present:  duflo
Word is not present:  elizabeth
Word is not present:  emily
Word is not present:  hilary
Word is not present:  hillary
Word is not present:  jane
Word is not present:  jennifer
Word is not present:  jessica
Word is not present:  maria
Word is not present:  mary
Word is not present:  nancy
Word is not present:

/var/folders/6m/7qj3x7j16pqbzfycpphbz9z40000gn/T/ipykernel_33101/142448549.py:20: RuntimeWarning: Mean of empty slice.
  averaged_words1 = np.array(words_to_average1).mean(axis=0)
/Users/bryskulov/Desktop/QEM Barcelona/Thesis/thesis-gender-bias/env/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/var/folders/6m/7qj3x7j16pqbzfycpphbz9z40000gn/T/ipykernel_33101/142448549.py:21: RuntimeWarning: Mean of empty slice.
  averaged_words2 = np.array(words_to_average2).mean(axis=0)
/var/folders/6m/7qj3x7j16pqbzfycpphbz9z40000gn/T/ipykernel_33101/3291762599.py:13: RuntimeWarning: Mean of empty slice.
  return np.array(list_rel_norm_dist).mean()


nan

In [52]:
calc_relative_norm_distance(w2v_model.wv, male_names_full, female_names, academic_general + professional)

Word is not present:  acemoglu
Word is not present:  adam
Word is not present:  akerlof
Word is not present:  alan
Word is not present:  albert
Word is not present:  alexander
Word is not present:  allen
Word is not present:  andrew
Word is not present:  angrist
Word is not present:  arthur
Word is not present:  autor
Word is not present:  baker
Word is not present:  barro
Word is not present:  becker
Word is not present:  ben
Word is not present:  benjamin
Word is not present:  bernanke
Word is not present:  berry
Word is not present:  blanchard
Word is not present:  bob
Word is not present:  borjas
Word is not present:  brian
Word is not present:  campbell
Word is not present:  carl
Word is not present:  carlos
Word is not present:  charlie
Word is not present:  chetty
Word is not present:  chris
Word is not present:  christopher
Word is not present:  cochrane
Word is not present:  colin
Word is not present:  cowen
Word is not present:  daniel
Word is not present:  daron
Word is not 

/var/folders/6m/7qj3x7j16pqbzfycpphbz9z40000gn/T/ipykernel_33101/142448549.py:21: RuntimeWarning: Mean of empty slice.
  averaged_words2 = np.array(words_to_average2).mean(axis=0)


nan

In [53]:
calc_relative_norm_distance(w2v_model.wv, male_names, female_names, personal_information + physical_attributes)

Word is not present:  acemoglu
Word is not present:  adam
Word is not present:  akerlof
Word is not present:  alan
Word is not present:  albert
Word is not present:  alexander
Word is not present:  allen
Word is not present:  andrew
Word is not present:  angrist
Word is not present:  arthur
Word is not present:  autor
Word is not present:  baker
Word is not present:  barro
Word is not present:  becker
Word is not present:  ben
Word is not present:  benjamin
Word is not present:  bernanke
Word is not present:  berry
Word is not present:  blanchard
Word is not present:  bob
Word is not present:  borjas
Word is not present:  amy
Word is not present:  athey
Word is not present:  duflo
Word is not present:  elizabeth
Word is not present:  emily
Word is not present:  hilary
Word is not present:  hillary
Word is not present:  jane
Word is not present:  jennifer
Word is not present:  jessica
Word is not present:  maria
Word is not present:  mary
Word is not present:  nancy
Word is not present:

/var/folders/6m/7qj3x7j16pqbzfycpphbz9z40000gn/T/ipykernel_33101/142448549.py:20: RuntimeWarning: Mean of empty slice.
  averaged_words1 = np.array(words_to_average1).mean(axis=0)
/var/folders/6m/7qj3x7j16pqbzfycpphbz9z40000gn/T/ipykernel_33101/142448549.py:21: RuntimeWarning: Mean of empty slice.
  averaged_words2 = np.array(words_to_average2).mean(axis=0)


nan

# 7. WEAT test by Caliscan et al. (2017)

## 7.1 Check baseline bias

In [54]:
# load the test model
model = load_test_model()

INFO - 19:25:37: loading KeyedVectors object from /Users/bryskulov/Desktop/QEM Barcelona/Thesis/thesis-gender-bias/env/lib/python3.9/site-packages/wefe/datasets/data/test_model.kv
INFO - 19:25:37: setting ignored attribute vectors_norm to None
INFO - 19:25:37: KeyedVectors lifecycle event {'fname': '/Users/bryskulov/Desktop/QEM Barcelona/Thesis/thesis-gender-bias/env/lib/python3.9/site-packages/wefe/datasets/data/test_model.kv', 'datetime': '2022-06-21T19:25:37.964800', 'gensim': '4.2.0', 'python': '3.9.10 (v3.9.10:f2f3f53782, Jan 13 2022, 17:02:14) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.16-x86_64-i386-64bit', 'event': 'loaded'}


In [55]:
family = ["home", "parents", "children", "family", "cousin", "marriage",
         "wedding"]
career = ["executive", "management", "professional", "corporation", "salary",
         "office", "business", "career"]

In [56]:
# define the query
query = Query(
    target_sets=[
        ["female", "woman", "girl", "sister", "she", "her", "hers",
         "daughter"],
        ["male", "man", "boy", "brother", "he", "him", "his", "son"],
    ],
    attribute_sets=[
        family, career
    ],
    target_sets_names=["Female terms", "Male Terms"],
    attribute_sets_names=["Family", "Career"],
)


# instance the metric and run the query
WEAT().run_query(query, model, calculate_p_value=True) 

{'query_name': 'Female terms and Male Terms wrt Family and Career',
 'result': 0.44263947810811377,
 'weat': 0.44263947810811377,
 'effect_size': 0.4196374793469268,
 'p_value': 0.2024797520247975}

## 7.2 Running WEAT tests 

In [263]:
wefe_model = WordEmbeddingModel(w2v_model.wv, 'AliceWu model')

### 7.2.1 Gender pronouns

In [385]:
# define the query
query_pronouns = Query(
    target_sets=[
        female_pronouns,
        male_pronouns,
    ],
    attribute_sets=[
        academic_general + professional,
        personal_information + physical_attributes
    ],
    target_sets_names=["Female pronouns", "Male pronouns"],
    attribute_sets_names=["Professional", "Personal"],
)

In [389]:
# instance the metric and run the query
WEAT().run_query(query_pronouns, wefe_model, normalize=True, calculate_p_value=True, 
                 p_value_method="approximate", p_value_iterations=10000, lost_vocabulary_threshold=0.5) 

{'query_name': 'Female pronouns and Male pronouns wrt Professional and Personal',
 'result': -0.7228623523066441,
 'weat': -0.7228623523066441,
 'effect_size': -1.8917259145115586,
 'p_value': 0.9875173370319001}

### 7.2.2 Gender names

In [387]:
# define the query
query_names = Query(
    target_sets=[
        female_names,
        male_names,
    ],
    attribute_sets=[
        academic_general + professional,
        personal_information + physical_attributes
    ],
    target_sets_names=["Female Names", "Male Names"],
    attribute_sets_names=["Professional", "Personal"],
)

In [388]:
# instance the metric and run the query
WEAT().run_query(query_names, wefe_model, normalize=True, calculate_p_value=True, 
                 p_value_method="approximate", p_value_iterations=10000) 

{'query_name': 'Female Names and Male Names wrt Professional and Personal',
 'result': -1.7645896530399718,
 'weat': -1.7645896530399718,
 'effect_size': -1.2123690220188075,
 'p_value': 0.9999000099990001}

### 7.2.3 All gender-identifier words

In [392]:
# define the query
query_words = Query(
    target_sets=[
        female_words,
        male_words,
    ],
    attribute_sets=[
        academic_general + professional,
        personal_information + physical_attributes
    ],
    target_sets_names=["Female Words", "Male Words"],
    attribute_sets_names=["Professional", "Personal"],
)

In [393]:
# instance the metric and run the query
WEAT().run_query(query_words, wefe_model, normalize=True, calculate_p_value=True, 
                 p_value_method="approximate", p_value_iterations=10000) 

{'query_name': 'Female Words and Male Words wrt Professional and Personal',
 'result': -0.49194356167895925,
 'weat': -0.49194356167895925,
 'effect_size': -0.499921185807828,
 'p_value': 0.8083191680831917}

## 7.3 Run WEAT test on career and family words

In [390]:
# define the query
query_carfam = Query(
    target_sets=[
        female_words,
        male_words,
    ],
    attribute_sets=[
        career,
        family
    ],
    target_sets_names=["Female terms", "Male Terms"],
    attribute_sets_names=["Career", "Family"],
)

In [391]:
# instance the metric and run the query
WEAT().run_query(query_carfam, wefe_model, normalize=True, calculate_p_value=True, 
                 p_value_method="approximate", p_value_iterations=10000) 

{'query_name': 'Female terms and Male Terms wrt Career and Family',
 'result': -0.39675114249500143,
 'weat': -0.39675114249500143,
 'effect_size': -0.7348252153789426,
 'p_value': 0.8983101689831017}